This file takes a while to run since it goes through each link, loads it and scrapes the necessary data. Since it takes a while I've added in some code which takes any previously fetched movie data and ensures those are not scraped again. 

In [9]:
import pickle
import os

MOVIE_LINKS_FILE = '2links24-12-2021-19-29-31-9457.pkl'
PREV_MOVIES_DATA_FILE = "test-1769.pkl"

test_links = ['https://www.rottentomatoes.com/m/rumble_2021',
 'https://www.rottentomatoes.com/m/hurt_2021',
 'https://www.rottentomatoes.com/m/back_to_the_outback']

test_links = pickle.load(open( os.path.join('data', MOVIE_LINKS_FILE), 'rb'))[:30]

print("Number of links", len(test_links))

Number of links 30


In [12]:
import time
import logging
from datetime import datetime
from selenium.webdriver.common.by import By
from concurrent import futures
from SeleniumDriver import getDriver
from joblib import Parallel, delayed, cpu_count
print('Cpu count', cpu_count())

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield {"chunk": lst[i:i + n], "chunk_number": i}
NOW_DATE_STRING = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")

def saveMovies(movies, chunk_number=0): 
    name_of_file = 'movies-{num_of_movies}-chunk-{chunk_number}-date-{date}.pkl'.format(num_of_movies=len(movies), date=NOW_DATE_STRING, chunk_number=chunk_number)
    with open(os.path.join('raw_data', name_of_file), 'wb') as f:
            pickle.dump(movies, f)

prev_movies_data_hash_map = {}
# if (PREV_MOVIES_DATA_FILE):
#     prev_movies_data = pickle.load(open(os.path.join('data',PREV_MOVIES_DATA_FILE), 'rb'))
#     for movie in prev_movies_data:
#         link = movie['link']
#         prev_movies_data_hash_map[link] = True

def scrapeMovies(test_links, chunk_number):
    print('First url' , test_links[0])
    driver = getDriver()
    scraped_movies = []
    errored_links = []

    for i in range(len(test_links)):
        link = test_links[i]
        print("i", i)
        # if i > 0 and (i % 500 == 0):
        saveMovies(scraped_movies, chunk_number)
        try:
            if (link in prev_movies_data_hash_map):
                continue

            print( "Scraping link", link)
            driver.get(link)
            time.sleep(0.2)

            # TODO Don't skip whole movie if some fields error
            score_board_elm = driver.find_element(By.TAG_NAME, "score-board")
            audience_score = score_board_elm.get_attribute("audiencescore")
            tomato_meter_score = score_board_elm.get_attribute("tomatometerscore")
            release_date = driver.find_element(By.TAG_NAME, "time").text
            page_title = driver.title
            information = {
                "audience_score" :  audience_score,
                "tomato_meter_score" :  tomato_meter_score,
                "release_date": release_date,
                "page_title": page_title,
                "link": link,
            }
            scraped_movies.append(information)
        except Exception as e:
            logging.error('bad link', link)
            # logging.exception(e, exc_info=True) # Uncomment for debugging, could use debug levels in future
            errored_links.append(link)

    saveMovies(scraped_movies)
    print('Number of errored movie links', len(errored_links))

# print(test_links)
# print([links for links in chunks(test_links, 5)])
Parallel(n_jobs=-1)(delayed(scrapeMovies)(chunk['chunk'], chunk['chunk_number']) for chunk in chunks(test_links,10)) #execute parallel for all urls


Cpu count 4
First url https://www.rottentomatoes.com/m/the_hating_game
First url First urlhttps://www.rottentomatoes.com/m/the_scary_of_sixty_first
 https://www.rottentomatoes.com/m/rumble_2021
i 0
Scraping link https://www.rottentomatoes.com/m/the_hating_game
i 0
Scraping link https://www.rottentomatoes.com/m/rumble_2021
i 0
Scraping link https://www.rottentomatoes.com/m/the_scary_of_sixty_first


KeyboardInterrupt: 